# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

In [ ]:
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
# !wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad

# !cat Dataset.tar.gz.part* > Dataset.tar.gz

# # unzip the file
# !tar zxvf Dataset.tar.gz

In [ ]:
import os
import random
import pandas as pd
import torch
import gc
gc.collect()
# os.environ["CUDA_VISIBLE_DEVICES"]='1'
torch.cuda.empty_cache()

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
!nvidia-smi

Thu Mar 17 08:46:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   52C    P2    42W / 180W |   1840MiB /  8192MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Fix Random Seed

In [ ]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
			return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.8 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [ ]:
# !pip3 install torchaudio

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchaudio


class Classifier(nn.Module):
	def __init__(self, d_model=200, n_spks=800, dropout=0.3):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		# self.encoder_layer = nn.TransformerEncoderLayer(
		# 	d_model=d_model, dim_feedforward=256, nhead=2
		# )
		self.encoder_layer = torchaudio.models.Conformer(input_dim = d_model, ffn_dim=256, num_heads=1, num_layers=1, depthwise_conv_kernel_size=31)#,
                                                   			# ff_dropout=dropout, atth_dropout=dropout, conv_dropout=dropout)
		# self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Dropout(p=0.3),
			nn.Linear(d_model, n_spks),
			# nn.Linear(d_model, d_model),
   			# nn.Dropout(p=0.3),
			# nn.LeakyReLU(),
			# nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# out: (batch size, length, d_model)
		out = self.prenet(mels)
		# out: (length, batch size, d_model)
		# out = out.permute(1, 0, 2)
		device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
		lengths = [len(i) for i in out]
		Lengths = torch.tensor(lengths)
		Lengths = Lengths.to(device)
		
  		# The encoder layer expect features in the shape of (length, batch size, d_model).
		out = self.encoder_layer(out, Lengths) ### add batch size lengths
		# out: (batch size, length, d_model)
		# print(out)
		out = out[0] ### tuple to data
		# out = out.transpose(0, 1)
		# mean pooling
		stats = out.mean(dim=1)

		# out: (batch, n_spks)
		out = self.pred_layer(stats)
		return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [ ]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	# print(len(mels),len(labels)) #32 32
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)
	# print(len(outs),len(labels)) #32 32

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [ ]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 1000000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()
			# print("valid_accuracy:",valid_accuracy,"\t best_accuracy:",best_accuracy)

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:24<00:00, 82.90 step/s, accuracy=0.25, loss=3.16, step=2000]
Valid: 100% 11328/11334 [00:02<00:00, 4413.63 uttr/s, accuracy=0.23, loss=3.65]
Train: 100% 2000/2000 [00:24<00:00, 81.98 step/s, accuracy=0.38, loss=3.12, step=4000]
Valid: 100% 11328/11334 [00:02<00:00, 3994.15 uttr/s, accuracy=0.36, loss=2.95]
Train: 100% 2000/2000 [00:24<00:00, 80.78 step/s, accuracy=0.50, loss=2.02, step=6000]
Valid: 100% 11328/11334 [00:02<00:00, 4505.76 uttr/s, accuracy=0.45, loss=2.43]
Train: 100% 2000/2000 [00:24<00:00, 80.85 step/s, accuracy=0.50, loss=1.85, step=8000]
Valid: 100% 11328/11334 [00:02<00:00, 3877.95 uttr/s, accuracy=0.52, loss=2.11]
Train: 100% 2000/2000 [00:25<00:00, 77.81 step/s, accuracy=0.66, loss=1.84, step=1e+4]
Valid: 100% 11328/11334 [00:03<00:00, 3624.17 uttr/s, accuracy=0.55, loss=1.96]
Train:   1% 14/2000 [00:00<00:29, 67.47 step/s, accuracy=0.56, loss=1.68, step=1e+4]

Step 10000, best model saved. (accuracy=0.5481)


Train: 100% 2000/2000 [00:24<00:00, 80.97 step/s, accuracy=0.59, loss=1.98, step=12000]
Valid: 100% 11328/11334 [00:02<00:00, 4397.95 uttr/s, accuracy=0.60, loss=1.77]
Train: 100% 2000/2000 [00:24<00:00, 80.73 step/s, accuracy=0.47, loss=2.01, step=14000]
Valid: 100% 11328/11334 [00:02<00:00, 3866.47 uttr/s, accuracy=0.62, loss=1.67]
Train: 100% 2000/2000 [00:25<00:00, 79.15 step/s, accuracy=0.72, loss=1.40, step=16000]
Valid: 100% 11328/11334 [00:02<00:00, 4474.47 uttr/s, accuracy=0.64, loss=1.54]
Train: 100% 2000/2000 [00:24<00:00, 80.28 step/s, accuracy=0.62, loss=1.38, step=18000]
Valid: 100% 11328/11334 [00:02<00:00, 4368.86 uttr/s, accuracy=0.65, loss=1.49]
Train: 100% 2000/2000 [00:25<00:00, 78.27 step/s, accuracy=0.84, loss=0.58, step=2e+4] 
Valid: 100% 11328/11334 [00:03<00:00, 3650.19 uttr/s, accuracy=0.67, loss=1.43]
Train:   1% 14/2000 [00:00<00:28, 69.33 step/s, accuracy=0.59, loss=2.05, step=2e+4]

Step 20000, best model saved. (accuracy=0.6682)


...

Step 980000, best model saved. (accuracy=0.9116)


Train: 100% 2000/2000 [00:22<00:00, 87.54 step/s, accuracy=1.00, loss=0.04, step=982000]
Valid: 100% 11328/11334 [00:02<00:00, 5264.55 uttr/s, accuracy=0.90, loss=0.46]
Train: 100% 2000/2000 [00:22<00:00, 88.22 step/s, accuracy=1.00, loss=0.02, step=984000]
Valid: 100% 11328/11334 [00:02<00:00, 5261.75 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [00:23<00:00, 86.76 step/s, accuracy=1.00, loss=0.02, step=986000]
Valid: 100% 11328/11334 [00:02<00:00, 5333.71 uttr/s, accuracy=0.90, loss=0.45]
Train: 100% 2000/2000 [00:22<00:00, 87.96 step/s, accuracy=0.97, loss=0.08, step=988000]
Valid: 100% 11328/11334 [00:02<00:00, 5175.78 uttr/s, accuracy=0.91, loss=0.44]
Train: 100% 2000/2000 [00:22<00:00, 87.06 step/s, accuracy=0.97, loss=0.07, step=990000]
Valid: 100% 11328/11334 [00:02<00:00, 5153.31 uttr/s, accuracy=0.91, loss=0.45]
Train:   1% 17/2000 [00:00<00:23, 84.14 step/s, accuracy=0.97, loss=0.09, step=990017]

Step 990000, best model saved. (accuracy=0.9116)


Train: 100% 2000/2000 [00:22<00:00, 88.31 step/s, accuracy=0.97, loss=0.07, step=992000]
Valid: 100% 11328/11334 [00:02<00:00, 5354.47 uttr/s, accuracy=0.91, loss=0.46]
Train: 100% 2000/2000 [00:22<00:00, 88.24 step/s, accuracy=1.00, loss=0.06, step=994000]
Valid: 100% 11328/11334 [00:02<00:00, 4836.29 uttr/s, accuracy=0.90, loss=0.45]
Train: 100% 2000/2000 [00:23<00:00, 85.47 step/s, accuracy=0.97, loss=0.13, step=996000]
Valid: 100% 11328/11334 [00:02<00:00, 5264.72 uttr/s, accuracy=0.91, loss=0.44]
Train: 100% 2000/2000 [00:22<00:00, 87.86 step/s, accuracy=1.00, loss=0.00, step=998000]
Valid: 100% 11328/11334 [00:02<00:00, 5259.18 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [00:22<00:00, 87.11 step/s, accuracy=1.00, loss=0.00, step=1e+6]  
Valid: 100% 11328/11334 [00:02<00:00, 5194.81 uttr/s, accuracy=0.91, loss=0.44]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 1000000, best model saved. (accuracy=0.9116)


# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)
	
	ans = []
	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			ans.append(preds)
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)
  	

if __name__ == "__main__":
	main(**parse_args())